# Imports

In [2]:
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import ParameterGrid
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
import datasets
from weighted_random_search import wrs
import tqdm
import matplotlib.pyplot as plt

In [4]:
import import_ipynb
from CNN import CNN_3_class

# Classes

In [5]:
class Net_wrapper:
    """ 
    Wrapper for neural network model. It combines the model itself (nn.Module) together with
    optimizer, loss function and training parameters (such as max_epochs, learning rate and batch size)
    """
        
    def __init__(self, model=CNN_3_class, criterion=nn.CrossEntropyLoss, optimizer=optim.Adam,
                 max_epochs=5, batch_size=32, learning_rate=0.001, step_size=10, gamma=0.5, tqdm=False, **kwargs):
        if kwargs:
            self.model_params = kwargs
        else:
            self.model_params = {}
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.max_epochs = max_epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.scheduler_step_size=step_size
        self.scheduler_gamma=gamma
    
    def __setattr__(self, name, value):
        self.__dict__[name] = value

    def score(self, train_dataset, val_dataset):
        """
        Train model on train_dataset and calculate validation acurracy on val_dataset. 
        """
        if self.model_params:
            model = self.model(**self.model_params)
            
        else:
            model = self.model()
            
        optimizer = self.optimizer(model.parameters(), lr=self.learning_rate)
        scheduler = StepLR(optimizer, step_size=self.scheduler_step_size, gamma=self.scheduler_gamma)
        
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=self.batch_size, shuffle=False)

        for epoch in range(self.max_epochs):
            train_accuracies = []
            train_losses = []
            for data in tqdm(train_loader):
                results = model.train_step(data, optimizer, self.criterion())
                train_accuracies.append(results['accuracy'].item())
                train_losses.append(results['loss'].item())
            # Calculate average training loss and accuracy for the epoch
            avg_train_accuracy = sum(train_accuracies) / len(train_accuracies)
            avg_train_loss = sum(train_losses) / len(train_losses)

            # Test the model
            val_losses = []
            val_accuracies = []
            
            with torch.no_grad():
                for data in tqdm(val_loader):
                    results = model.test_step(data, self.criterion())
                    val_losses.append(results['loss'].item())
                    val_accuracies.append(results['accuracy'].item())

            # Calculate average test loss and accuracy for the epoch
            avg_validation_accuracy = sum(val_accuracies) / len(val_accuracies)
            avg_validation_loss = sum(val_losses) / len(val_losses)

            scheduler.step()

        return avg_train_accuracy, avg_train_loss, avg_validation_accuracy, avg_vallidation_loss
        
    

In [6]:
class GridSearch():
    """
    Class used to perform grid search on neural networks 

    Attributes:
    self.net - Net_wrapper instance
    self.param_grid - dictionary of parameters we want to search
    self.scores - list for scores of each set of parameters
    self.best_score - best score out of all parameters
    self.best_params - best set of parameters
    self.verbose - if set to 1 additional information (parameter set and accuracy) prints with each iteration of grid search. 
    """
    def __init__(self, net: Net_wrapper, param_grid, step_by_step=False, verbose=1):
        """

        """
        self.net = net
        self.param_grid = param_grid
        self.scores = []
        self.best_score = 0
        self.best_params = {}
        self.step_by_step = step_by_step
        self.verbose = verbose


    def fit(self, train_dataset, val_dataset):
        """
        Fit the grid search with train and validation dataset. 
        Search for optimal parameters for neural network declared during 
        initialization of GridSearch instance.
        """
        if self.step_by_step==False:
            for params in ParameterGrid(self.param_grid):
                for hyp_name, hyp_val in params.items():
                    if hasattr(self.net, hyp_name):
                        setattr(self.net, hyp_name, hyp_val)
                    else:
                        self.net.model_params[hyp_name] = hyp_val

                _, val_accuracy = self.net.score(train_dataset, val_dataset)
                self.scores.append(val_accuracy)
                if val_accuracy > self.best_score:
                    self.best_score = val_accuracy
                    self.best_params = params
                if self.verbose == 1:
                    print('Parameter set:', params)
                    print(f'val_accuracy: {val_accuracy:.4f}')
        else:
            
            for hyp_name, hyp_vals in self.param_grid.items():
                score = 0
                for hyp_val in hyp_vals:
                    if hasattr(self.net, hyp_name):
                        setattr(self.net, hyp_name, hyp_val)
                    else:
                        self.net.model_params[hyp_name] = hyp_val 

                    _, val_accuracy = self.net.score(train_dataset, val_dataset)
                    self.scores.append(val_accuracy)
                    
                    if val_accuracy > score:
                        if score > self.best_score:
                            self.best_score = score
                        score = val_accuracy
                        self.best_params[hyp_name] = hyp_val
                    
                    if self.verbose == 1:
                        print(f'Current parameter: {hyp_name}:', hyp_val, f'  val_accuracy: {val_accuracy:.4f}')
                        print(f'Best parameters till now:{self.best_params}')
                
                if hasattr(self.net, hyp_name):
                    setattr(self.net, hyp_name, self.best_params[hyp_name])
                    print(getattr(self.net, hyp_name))
                else:
                    self.net.model_params[hyp_name] = self.best_params[hyp_name]

        return self

In [6]:
class WeightedRandomSearch():
    """
    Class used to perform grid search on neural networks 

    Attributes:
    self.net - Net_wrapper instance
    self.param_grid - dictionary of parameters we want to search
    self.scores - list for scores of each set of parameters
    self.best_score - best score out of all parameters
    self.best_params - best set of parameters
    self.verbose - if set to 1 additional information (parameter set and accuracy) prints with each iteration of grid search. 
    """
    def __init__(self, net, param_grid, verbose=1):
        """

        """
        self.net = net
        self.param_grid = param_grid
        self.scores = []
        self.best_score = 0
        self.best_params = None
        self.verbose = verbose

    def fit(self, train_dataset, val_dataset, N, N_0):
        """
        Fit the grid search with train and validation dataset. 
        Search for optimal parameters for neural network declared during 
        initialization of GridSearch instance.
        """
        def goal_function(params):
            
            for hyp_name, hyp_val in params.items():
                if hasattr(self.net, hyp_name):
                    setattr(self.net, hyp_name, hyp_val)
                else:
                    self.net.model_params[hyp_name] = hyp_val
            
            return self.net.score(train_dataset, val_dataset)[1]
            
        self.best_params, self.best_score = wrs(F=goal_function, N=N, N_0=N_0, param_grid=self.param_grid )
        
        return self

### Example hyperparameters (from most to least important):

In [6]:
hyperparams1 = {'learning_rate': [0.001, 0.005, 0.01],
               'batch_size': [8, 32, 64],
               'max_epochs': [5, 10, 15]}

hyperparams2 = {
                'no_neurons': [25, 50, 100],
                'kernel_size': [2, 3, 5],
                'number_of_filters': [16, 32, 64]
                }

hyperparams3 = {
               'optimizer': [optim.Adam, optim.Adagrad, optim.SGD],
               'activation_func': [nn.ReLU, nn.LeakyReLU, nn.Tanh],
               'dropout_rate': [0.0, 0.25, 0.5]}



### Sample for testing GridSearch

In [7]:
train_dataset = datasets.cifar_train
val_dataset = datasets.cifar_val

In [7]:
from torch.utils.data import DataLoader, SubsetRandomSampler, Subset

subset_indices = list(range(500))
subset_sampler = SubsetRandomSampler(subset_indices)

subset_train_dataset = Subset(train_dataset, subset_indices)
subset_val_dataset = Subset(val_dataset, subset_indices)

### GridSearch test

In [15]:
test_hyper_params = {'learning_rate': [0.0001, 0.0005, 0.001, 0.005, 0.01], 'batch_size': [8, 16, 32, 64, 128]}
my_net = Net_wrapper(model=CNN_3_class, max_epochs=5)
gs = GridSearch(net=my_net, param_grid=test_hyper_params, step_by_step=True, verbose=1)
gs = gs.fit(subset_train_dataset, subset_val_dataset)

Current parameter: learning_rate: 0.0001   val_accuracy: 0.2344
Best parameters till now:{'learning_rate': 0.0001}
Current parameter: learning_rate: 0.0005   val_accuracy: 0.3148
Best parameters till now:{'learning_rate': 0.0005}
Current parameter: learning_rate: 0.001   val_accuracy: 0.3156
Best parameters till now:{'learning_rate': 0.001}
Current parameter: learning_rate: 0.005   val_accuracy: 0.2574
Best parameters till now:{'learning_rate': 0.001}
Current parameter: learning_rate: 0.01   val_accuracy: 0.1008
Best parameters till now:{'learning_rate': 0.001}
0.001
Current parameter: batch_size: 8   val_accuracy: 0.3313
Best parameters till now:{'learning_rate': 0.001, 'batch_size': 8}
Current parameter: batch_size: 16   val_accuracy: 0.3418
Best parameters till now:{'learning_rate': 0.001, 'batch_size': 16}
Current parameter: batch_size: 32   val_accuracy: 0.3164
Best parameters till now:{'learning_rate': 0.001, 'batch_size': 16}
Current parameter: batch_size: 64   val_accuracy: 0.3

In [31]:
print(gs.best_score)
print(gs.best_params)

0.32341269841269843
{'learning_rate': 0.001, 'batch_size': 8, 'no_neurons': 128}


### Weighted random search test

In [28]:
my_net = Net_wrapper()
param_grid = {'lr': [0.01, 0.05, 0.1, 0.5, 1, 2], 'batch_size': [16, 32, 64, 128, 256]}
w = WeightedRandomSearch(net=my_net, param_grid=test_hyper_params)
N, N_0 = 10, 3
w.fit(subset_train_dataset, subset_val_dataset, N, N_0)
print(w.best_params)

{'learning_rate': 0.01, 'batch_size': 64, 'no_neurons': 64} 0.11944110598415136
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303


# Hyperparameter Tuning

## CNN_3_class on original dataset (without augmentation)

1. Learning rate + batch size

In [16]:
my_net = Net_wrapper(model=CNN_3_class, max_epochs=5)
param_grid = {'learning_rate': [0.0001, 0.0005, 0.001, 0.005, 0.01], 'batch_size': [8, 16, 32, 64, 128]}
gs = GridSearch(net=my_net, param_grid=param_grid, step_by_step=True, verbose=1)
gs = gs.fit(train_dataset, val_dataset)

Current parameter: learning_rate: 0.0001   val_accuracy: 0.5830
Best parameters till now:{'learning_rate': 0.0001}
Current parameter: learning_rate: 0.0005   val_accuracy: 0.6887
Best parameters till now:{'learning_rate': 0.0005}
Current parameter: learning_rate: 0.001   val_accuracy: 0.6761
Best parameters till now:{'learning_rate': 0.0005}
Current parameter: learning_rate: 0.005   val_accuracy: 0.5991
Best parameters till now:{'learning_rate': 0.0005}
Current parameter: learning_rate: 0.01   val_accuracy: 0.1000
Best parameters till now:{'learning_rate': 0.0005}
0.0005
Current parameter: batch_size: 8   val_accuracy: 0.6954
Best parameters till now:{'learning_rate': 0.0005, 'batch_size': 8}
Current parameter: batch_size: 16   val_accuracy: 0.6823
Best parameters till now:{'learning_rate': 0.0005, 'batch_size': 8}
Current parameter: batch_size: 32   val_accuracy: 0.6785
Best parameters till now:{'learning_rate': 0.0005, 'batch_size': 8}
Current parameter: batch_size: 64   val_accuracy

In [ ]:
learning_rate = [0.583, 0.6887, 0.6761, 0.5991, 0.1]
batch_size = [0.6954, 0.6823, 0.6785, 0.6427, 0.6242]

fig, ax = plt.subplots(1, 2, figsize=(12, 8))
ax[1].plot(param_grid['learning_rate'], learning_rate)
ax[2].plot(param_grid['batch_size'], batch_size)
plt.show()

Check if smaller batch size yields even better performance

In [8]:
my_net = Net_wrapper(model=CNN_3_class, max_epochs=5, learning_rate=0.005)
param_grid = {'batch_size': [1, 2, 4, 8]}
gs = GridSearch(net=my_net, param_grid=param_grid, step_by_step=True, verbose=1)
gs = gs.fit(train_dataset, val_dataset)

2. Num of epochs (run a lot of epochs and stop when validation accuracy starts to decrease)

In [ ]:
my_net = Net_wrapper(model=CNN_3_class, max_epochs=20, learning_rate=0.005, batch_size=8)
my_net.score(train_dataset, val_dataset)

3. 